<a href="https://colab.research.google.com/github/MominaGull/ML-Practice-Projects/blob/main/Accident_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing required libraries
import torch
from torchvision import transforms, utils
from torch.utils.data.dataset import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn import preprocessing

In [ ]:
# Mounting google drive to import dataset
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AH78T6sqZmgu_XjwRmqT9w61ZxxfdmP6Il6WSO05u0A-ULYFs0YERA
Mounted at /content/drive


In [ ]:
# Method to get images from drive using image path and labels from csv file
class ExtractDataFromCsv(Dataset):
    def __init__(self, csv_path):
        """
        Reading image locations and labels from csv
        but reading images from files
        Args:
            csv_path (string): path to csv file
        Returns:
            Image as transformed tensor and label
        """
        # Transforms images to gray scale and in 28 by 28 dimensions     
        self.to_tensor =  transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.Resize([28,28]) ,transforms.ToTensor()])

        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Third column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 2])
     
        # Calculate length of data in csv file
        self.data_len = len(self.data_info.index)

    # Function to get images and labels
    def __getitem__(self, index):
        # Get image name from the csv file
        single_image_name = self.image_arr[index]
        # Open image using path, folder name and image name
        img_as_img = Image.open('/content/drive/My Drive/vehicles/train/images/'  + single_image_name)

        # Apply tranformations on image stored in to_tensor
        img_as_tensor = self.to_tensor(img_as_img)
        # Get label of the image
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    # Fucntion to get length of data in csv file
    def __len__(self):
        return self.data_len

In [ ]:
# Read training image locations from Csv - transformations are defined inside ExtractDataFromCsv class
train_set = ExtractDataFromCsv('/content/drive/My Drive/vehicles/train/train.csv')

# Passing extracted images and labels to pytoch dataloader with batch size of 150
trainset_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=150, shuffle=False)

In [ ]:
# Creating CNN model
class Model(torch.nn.Module):
  #
  #Passing gray scale image. Input channel for first convulation layer should be one.
  #Two classes. Final output neurons should be 2.
  #
    def __init__(self):
        super(Model, self).__init__()
        self.conv_1 = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv_2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.max_pool2d = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear_1 = torch.nn.Linear(7 * 7 * 64, 128)
        self.linear_2 = torch.nn.Linear(128, 2)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.conv_1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv_2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        pred = self.linear_2(x)

        return pred       

In [ ]:
# Calling the Model class to get an object of CNN model
model = Model()
# Using CrossEntropyLoss to calculate loss
criterion = torch.nn.CrossEntropyLoss()
# Using Adam Optimizing algorithm with learning rate of 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training model for 100 epochs
no_epochs = 100
train_loss = list()

# Looping over total number of epochs
for epoch in range(no_epochs):
    # Setting initial total training loss to zero
    total_train_loss = 0

    # Iterating over images and labels in train dataloader object
    for itr, (images, labels) in enumerate(trainset_loader):
        
      # Transforming labels to tensors
      labels = torch.tensor(labels, dtype=torch.long)

      # Setting gradients to zero
      optimizer.zero_grad()

      # Passing image to CNN model
      pred = model(images)

      # Calculating loss 
      loss = criterion(pred, labels)
      # Updating training loss variable according to loss calculated
      total_train_loss += loss.item()
       
      # Performing back propagation
      loss.backward()
      optimizer.step()
  
    # Calculating total training loss
    total_train_loss = total_train_loss / (itr + 1)
    train_loss.append(total_train_loss)
 
    # Printing number of completed epochs, total epochs and training loss
    print('\nEpoch: {}/{}, Training Loss: {:.8f},'.format(epoch + 1, no_epochs, total_train_loss))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  



Epoch: 1/100, Training Loss: 2.45704769,

Epoch: 2/100, Training Loss: 0.67752889,

Epoch: 3/100, Training Loss: 0.69301741,

Epoch: 4/100, Training Loss: 0.69323463,

Epoch: 5/100, Training Loss: 0.69317179,

Epoch: 6/100, Training Loss: 0.69307300,

Epoch: 7/100, Training Loss: 0.69306733,

Epoch: 8/100, Training Loss: 0.69304054,

Epoch: 9/100, Training Loss: 0.69301738,

Epoch: 10/100, Training Loss: 0.69297167,

Epoch: 11/100, Training Loss: 0.69295166,

Epoch: 12/100, Training Loss: 0.69290478,

Epoch: 13/100, Training Loss: 0.69295638,

Epoch: 14/100, Training Loss: 0.69283739,

Epoch: 15/100, Training Loss: 0.69285975,

Epoch: 16/100, Training Loss: 0.69280791,

Epoch: 17/100, Training Loss: 0.69279794,

Epoch: 18/100, Training Loss: 0.69272490,

Epoch: 19/100, Training Loss: 0.69266707,

Epoch: 20/100, Training Loss: 0.69259818,

Epoch: 21/100, Training Loss: 0.69257933,

Epoch: 22/100, Training Loss: 0.69261953,

Epoch: 23/100, Training Loss: 0.69259503,

Epoch: 24/100, Trai

In [ ]:
# Read validation image locations from Csv - transformations are defined inside ExtractDataFromCsv class
val_set = ExtractDataFromCsv('/content/drive/My Drive/vehicles/validation/validation.csv')

# Passing extracted images and labels to pytoch dataloader with batch size of 150
valset_loader = torch.utils.data.DataLoader(dataset=val_set, batch_size=150, shuffle=False)

In [ ]:
# Validation model

results = list()
total = 0

# Iterating over images and labels in validation dataloader object
for itr, (image, label) in enumerate(valset_loader):
  
    # Passing image to model
    pred = model(image)
    # Predicting label for passed image
    pred = torch.nn.functional.softmax(pred, dim=1)

    # Looping over the predicted probabilities for both classes
    for i, p in enumerate(pred):

        # Printing predictions
        print("p.data:",p.data)
        # Printing actual labels
        print("label", label[i])

        # Condition to check if label is equal to prediction
        if label[i] == torch.max(p.data, 0)[1]:
            # Increment total by one if prediction is correct
            total = total + 1

            # Appending image and label in result list
            results.append((image, torch.max(p.data, 0)[1]))

# Calculating accuracy 
test_accuracy = total / (itr + 1)
# Printing accuaracy of model on validation set
print('Test accuracy {:.8f}'.format(test_accuracy))

p.data: tensor([0.5283, 0.4717])
label tensor(0)
p.data: tensor([0.7494, 0.2506])
label tensor(0)
p.data: tensor([0.7370, 0.2630])
label tensor(0)
p.data: tensor([0.5720, 0.4280])
label tensor(0)
p.data: tensor([0.5721, 0.4279])
label tensor(0)
p.data: tensor([0.8259, 0.1741])
label tensor(0)
p.data: tensor([0.9431, 0.0569])
label tensor(0)
p.data: tensor([0.9102, 0.0898])
label tensor(0)
p.data: tensor([0.6883, 0.3117])
label tensor(0)
p.data: tensor([0.8410, 0.1590])
label tensor(0)
p.data: tensor([0.8212, 0.1788])
label tensor(0)
p.data: tensor([0.5033, 0.4967])
label tensor(0)
p.data: tensor([0.8822, 0.1178])
label tensor(0)
p.data: tensor([0.3598, 0.6402])
label tensor(0)
p.data: tensor([0.6639, 0.3361])
label tensor(0)
p.data: tensor([0.9221, 0.0779])
label tensor(0)
p.data: tensor([0.8384, 0.1616])
label tensor(0)
p.data: tensor([0.6969, 0.3031])
label tensor(0)
p.data: tensor([0.8199, 0.1801])
label tensor(0)
p.data: tensor([0.7266, 0.2734])
label tensor(0)
p.data: tensor([0.46

In [ ]:
# Function to get predictions for an image
# 0 = car is not damaged
# 1 = car is damaged

def predict(image):
      """
        Args:
            image: image to classify
        Returns:
            Image prediction as damaged or not damaged
        """
  with torch.no_grad():

    # Transforms image to grascale, 28 by 28 dimension and tensor
    img_as_tensor = transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.Resize([28,28]) ,transforms.ToTensor()])(image)

    # Sets batch number for pytorch conv layer to none
    img_as_tensor = img_as_tensor[None]

    # Pass image tensor to model
    pred = model(img_as_tensor)
    # Get the prediction for the image
    pred = torch.nn.functional.softmax(pred, dim=1)
    prediction = np.argmax(pred, axis=1)

    # Return prediction class
    return prediction.item()

tensor([[0.6225, 0.3775]])
0


In [ ]:
#Saving the CNN architecture in a file
torch.save(model.state_dict(), 'model.pt')
print("Model Saved")

Model Saved
